## **Base de datos ChEMBL**

## **Instalando librerías**

In [1]:
!pip install chembl_webresource_client

## **Importando librerías**

In [2]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Búsqueda del target**

### **Serotonine**

In [3]:
target = new_client.target
target_query = target.search('alpha-synuclein')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P37840', 'xref_name': None, 'xre...",Homo sapiens,Alpha-synuclein,23.0,False,CHEMBL6152,"[{'accession': 'P37840', 'component_descriptio...",SINGLE PROTEIN,9606
1,[],Mus musculus,Alpha-synuclein,23.0,False,CHEMBL5169100,"[{'accession': 'O55042', 'component_descriptio...",SINGLE PROTEIN,10090
2,[],Homo sapiens,Synphilin-1,22.0,False,CHEMBL1926494,"[{'accession': 'Q9Y6H5', 'component_descriptio...",SINGLE PROTEIN,9606
3,"[{'xref_id': 'O00418', 'xref_name': None, 'xre...",Homo sapiens,Serine/threonine-protein kinase EEF2K,13.0,False,CHEMBL5026,"[{'accession': 'O00418', 'component_descriptio...",SINGLE PROTEIN,9606
4,[],Rattus norvegicus,Eukaryotic elongation factor 2 kinase,13.0,False,CHEMBL3325307,"[{'accession': 'P70531', 'component_descriptio...",SINGLE PROTEIN,10116
...,...,...,...,...,...,...,...,...,...
1408,[],Homo sapiens,26S proteasome,1.0,False,CHEMBL2364701,"[{'accession': 'Q99460', 'component_descriptio...",PROTEIN COMPLEX,9606
1409,[],Cavia porcellus,Natriuretic peptides A,1.0,False,CHEMBL3097985,"[{'accession': 'P27596', 'component_descriptio...",SINGLE PROTEIN,10141
1410,[],Homo sapiens,Voltage-gated potassium channel subunit Kv7.1/...,1.0,False,CHEMBL3430890,"[{'accession': 'P51787', 'component_descriptio...",PROTEIN COMPLEX,9606
1411,[],Homo sapiens,Cardiac myosin,1.0,False,CHEMBL3831286,"[{'accession': 'P12883', 'component_descriptio...",PROTEIN COMPLEX,9606


### **Seleccionando y extrayendo la data de bioactividad**

In [4]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL6152'

Obteniendo la data de bioactividad y cargándola en DataFrame

In [5]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [6]:
df = pd.DataFrame.from_dict(res)

In [7]:
df

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,10978896,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,26000.0
1,None,None,10978897,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,26000.0
2,None,None,10978898,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,16000.0
3,None,None,10978899,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,507.1
4,None,None,10978900,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,1440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24763208,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5124371,Inhibition of alpha-Synuclein (unknown origin)...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,uM,UO_0000065,None,5.44
89,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24763209,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5124371,Inhibition of alpha-Synuclein (unknown origin)...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,uM,UO_0000065,None,2.58
90,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24763210,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5124371,Inhibition of alpha-Synuclein (unknown origin)...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,uM,UO_0000065,None,3.93
91,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24835434,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5144380,Inhibition of alpha-syn (unknown origin) aggre...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,uM,UO_0000065,None,10.5


In [8]:
df['assay_chembl_id'].value_counts()

CHEMBL2065090    19
CHEMBL4700551    19
CHEMBL4700552    12
CHEMBL4625545     7
CHEMBL4342364     6
CHEMBL4700550     6
CHEMBL4334285     4
CHEMBL4700553     4
CHEMBL4700554     4
CHEMBL4700565     4
CHEMBL5124371     4
CHEMBL4252521     2
CHEMBL5144380     2
Name: assay_chembl_id, dtype: int64

In [23]:
df['assay_description'].value_counts()

Binding affinity to human alpha-synuclein expressed in Escherichia coli BL21 (DE3) cells after 1 hr by thioflavin T fluorescence assay                                                                  19
Inhibition of human alpha-synuclein filament formation expressed in Escherichia coli BL21(DE3) cells incubated for 72 hrs by thioflavin S based fluorescence assay                                      19
Inhibition of alpha-synuclein fibril formation (unknown origin) incubated for 24 hrs to 7 days by thioflavin S based fluorescence assay                                                                 12
Inhibition of alpha-synuclein (unknown origin) aggregation incubated for 3 days by thioflavin T based fluorescence assay                                                                                 7
Inhibition of alpha-synuclein aggregation (unknown origin) expressed in Escherichia coli BL21 (DE3) incubated for 3 days by thioflavin T fluorescence assay                                 

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 46 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   action_type                55 non-null     object
 1   activity_comment           0 non-null      object
 2   activity_id                93 non-null     int64 
 3   activity_properties        93 non-null     object
 4   assay_chembl_id            93 non-null     object
 5   assay_description          93 non-null     object
 6   assay_type                 93 non-null     object
 7   assay_variant_accession    0 non-null      object
 8   assay_variant_mutation     0 non-null      object
 9   bao_endpoint               93 non-null     object
 10  bao_format                 93 non-null     object
 11  bao_label                  93 non-null     object
 12  canonical_smiles           93 non-null     object
 13  data_validity_comment      3 non-null      object
 14  data_validit

In [10]:
df.shape

(93, 46)

Guardando en **bioactivity_data.csv**.

In [11]:
#df.to_csv('bioactivity_data_raw.csv', index=False)

## **Datos perdidos**
Si hay datos perdidos en las columnas **standard_value** y **canonical_smiles** estos son dropeados.

In [12]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,10978896,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,26000.0
1,None,None,10978897,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,26000.0
2,None,None,10978898,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,16000.0
3,None,None,10978899,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,507.1
4,None,None,10978900,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,1440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24763208,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5124371,Inhibition of alpha-Synuclein (unknown origin)...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,uM,UO_0000065,None,5.44
89,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24763209,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5124371,Inhibition of alpha-Synuclein (unknown origin)...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,uM,UO_0000065,None,2.58
90,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24763210,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5124371,Inhibition of alpha-Synuclein (unknown origin)...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,uM,UO_0000065,None,3.93
91,"{'action_type': 'INHIBITOR', 'description': 'N...",None,24835434,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5144380,Inhibition of alpha-syn (unknown origin) aggre...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,uM,UO_0000065,None,10.5


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93 entries, 0 to 92
Data columns (total 46 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   action_type                55 non-null     object
 1   activity_comment           0 non-null      object
 2   activity_id                93 non-null     int64 
 3   activity_properties        93 non-null     object
 4   assay_chembl_id            93 non-null     object
 5   assay_description          93 non-null     object
 6   assay_type                 93 non-null     object
 7   assay_variant_accession    0 non-null      object
 8   assay_variant_mutation     0 non-null      object
 9   bao_endpoint               93 non-null     object
 10  bao_format                 93 non-null     object
 11  bao_label                  93 non-null     object
 12  canonical_smiles           93 non-null     object
 13  data_validity_comment      3 non-null      object
 14  data_validit

In [14]:
len(df2.canonical_smiles.unique())

87

In [15]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr.head()

,action_type,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,None,10978896,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,26000.0
1,None,None,10978897,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,26000.0
2,None,None,10978898,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,16000.0
3,None,None,10978899,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,507.1
4,None,None,10978900,[],CHEMBL2065090,Binding affinity to human alpha-synuclein expr...,B,None,None,BAO_0000190,...,Homo sapiens,Alpha-synuclein,9606,None,None,IC50,nM,UO_0000065,None,1440.0


## **Pre-procesamiento de los datos de bioactividad**

### **Combinar las 3 columnas (molecule_chembl_id,canonical_smiles,standard_value) y bioactivity_class en el DataFrame**

In [16]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3.head()

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL828,c1ccc2c(c1)Nc1ccccc1S2,26000.0
1,CHEMBL1629795,CN(C)CCC(=O)NN1c2ccccc2Sc2cc(Cl)ccc21,26000.0
2,CHEMBL3142202,CN1CCN(NCCN2c3ccccc3Sc3ccccc32)CC1.Cl.Cl,16000.0
3,CHEMBL2063785,COc1ccc2c(c1)Sc1cc(OC)ccc1N2,507.1
4,CHEMBL2063786,COc1ccc2c(c1)Sc1cc(C#N)ccc1N2,1440.0


Guardar dataframe como arvhico CSV

In [17]:
df3.to_csv('Bioactivity_data_preprocessed.csv', index=False)

### **Etiquetando compuestos como activos, inactivos o intermedios**
La bioactividad se encuentra en unidades de IC50. Los compuestos teniendo valores menores que 1000 nM serán consideradas **active** mientas que mayores a 10 000 nM serán considerados como **inactive**. Valores entre 1,000 y 10,000 nM serán etiquetados como **intermediate**.

In [18]:
df4 = pd.read_csv('Bioactivity_data_preprocessed.csv')

In [19]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [20]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

,molecule_chembl_id,canonical_smiles,standard_value,class
0,CHEMBL828,c1ccc2c(c1)Nc1ccccc1S2,26000.0,inactive
1,CHEMBL1629795,CN(C)CCC(=O)NN1c2ccccc2Sc2cc(Cl)ccc21,26000.0,inactive
2,CHEMBL3142202,CN1CCN(NCCN2c3ccccc3Sc3ccccc32)CC1.Cl.Cl,16000.0,inactive
3,CHEMBL2063785,COc1ccc2c(c1)Sc1cc(OC)ccc1N2,507.1,active
4,CHEMBL2063786,COc1ccc2c(c1)Sc1cc(C#N)ccc1N2,1440.0,intermediate
...,...,...,...,...
82,CHEMBL5199332,O=C(Nc1ccc2oc3ccc(NC(=O)c4ccc(O)c(O)c4)cc3c(=O...,5440.0,intermediate
83,CHEMBL5176215,O=C(/C=C/c1cc(O)c(O)c(O)c1)Nc1ccc2oc3ccc(NC(=O...,2580.0,intermediate
84,CHEMBL5176552,O=C(/C=C/c1cc(O)ccc1O)Nc1ccc2oc3ccccc3c(=O)c2c1,3930.0,intermediate
85,CHEMBL5169662,O=C(Nc1ccc(NC(=O)C2CCNCC2)cc1)Nc1nc2ccc(O)cc2s1,10500.0,inactive


Saves dataframe to CSV file

In [21]:
df5.to_csv('Bioactivity_data_curated.csv', index=False)

In [22]:
! zip Bioactividad_df5.zip *.csv

  adding: Bioactivity_data_curated.csv (deflated 75%)
  adding: Bioactivity_data_preprocessed.csv (deflated 72%)


---